# HCT Survival Predictions

Goal:  Develop models to improve the prediction of transplant survival rates for patients undergoing allogeneic Hematopoietic Cell Transplantation (HCT) — an important step in ensuring that every patient has a fair chance at a successful outcome, regardless of their background.

Improving survival predictions for allogeneic HCT patients is a vital healthcare challenge. Current predictive models often fall short in addressing disparities related to socioeconomic status, race, and geography. Addressing these gaps is crucial for enhancing patient care, optimizing resource utilization, and rebuilding trust in the healthcare system.

The goal is to address disparities by bridging diverse data sources, refining algorithms, and reducing biases to ensure equitable outcomes for patients across diverse race groups. Your work will help create a more just and effective healthcare environment, ensuring every patient receives the care they deserve.

Dataset Description

The dataset consists of 59 variables related to hematopoietic stem cell transplantation (HSCT), encompassing a range of demographic and medical characteristics of both recipients and donors, such as age, sex, ethnicity, disease status, and treatment details. The primary outcome of interest is event-free survival, represented by the variable efs, while the time to event-free survival is captured by the variable efs_time. These two variables together encode the target for a censored time-to-event analysis. The data, which features equal representation across recipient racial categories including White, Asian, African-American, Native American, Pacific Islander, and More than One Race, was synthetically generated using the data generator from synthcity, trained on a large cohort of real CIBMTR data.


    train.csv - the training set, with target efs (Event-free survival)
    test.csv - the test set; your task is to predict the value of efs for this data
    sample_submission.csv - a sample submission file in the correct format with all predictions set to 0.50
    data_dictionary.csv - a list of all features and targets used in dataset and their descriptions


## Import Package

In [1]:
import numpy as np
import pandas as pd

## Import Dataset

In [2]:
path_data_dictionary = "C:/Users/julia/Desktop/Yanjun/hct competition/data_dictionary.csv"
path_test = "C:/Users/julia/Desktop/Yanjun/hct competition/test.csv"
path_train = "C:/Users/julia/Desktop/Yanjun/hct competition/train.csv"
data_dictionary = pd.read_csv(path_data_dictionary)
path_submission = "C:/Users/julia/Desktop/Yanjun/hct competition/sample_submission.csv"
test = pd.read_csv(path_test)
train = pd.read_csv(path_train)
submission = pd.read_csv(path_submission)

In [3]:
data_dictionary

,variable,description,type,values
0,dri_score,Refined disease risk index,Categorical,['Intermediate' 'High' 'N/A - non-malignant in...
1,psych_disturb,Psychiatric disturbance,Categorical,['Yes' 'No' nan 'Not done']
2,cyto_score,Cytogenetic score,Categorical,['Intermediate' 'Favorable' 'Poor' 'TBD' nan '...
3,diabetes,Diabetes,Categorical,['No' 'Yes' nan 'Not done']
4,hla_match_c_high,Recipient / 1st donor allele level (high resol...,Numerical,NaN
5,hla_high_res_8,Recipient / 1st donor allele-level (high resol...,Numerical,NaN
6,tbi_status,TBI,Categorical,"['No TBI' 'TBI + Cy +- Other' 'TBI +- Other, <..."
7,arrhythmia,Arrhythmia,Categorical,['No' nan 'Yes' 'Not done']
8,hla_low_res_6,Recipient / 1st donor antigen-level (low resol...,Numerical,NaN
9,graft_type,Graft type,Categorical,['Peripheral blood' 'Bone marrow']


In [4]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28800 entries, 0 to 28799
Data columns (total 60 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   ID                      28800 non-null  int64  
 1   dri_score               28646 non-null  object 
 2   psych_disturb           26738 non-null  object 
 3   cyto_score              20732 non-null  object 
 4   diabetes                26681 non-null  object 
 5   hla_match_c_high        24180 non-null  float64
 6   hla_high_res_8          22971 non-null  float64
 7   tbi_status              28800 non-null  object 
 8   arrhythmia              26598 non-null  object 
 9   hla_low_res_6           25530 non-null  float64
 10  graft_type              28800 non-null  object 
 11  vent_hist               28541 non-null  object 
 12  renal_issue             26885 non-null  object 
 13  pulm_severe             26665 non-null  object 
 14  prim_disease_hct        28800 non-null

In [5]:
# remove Id from train
train1 = train.copy()
train1 = train1.drop(columns = 'ID')

In [6]:
train1.efs.value_counts('percent')

# efs = 1 means that this patient has Event after transplant surgery
# efs = 0 means that this patient doesn't has Event after transplant surgery

efs
1.0    0.539306
0.0    0.460694
Name: proportion, dtype: float64

In [7]:
train1.head()

,dri_score,psych_disturb,cyto_score,diabetes,hla_match_c_high,hla_high_res_8,tbi_status,arrhythmia,hla_low_res_6,graft_type,...,tce_div_match,donor_related,melphalan_dose,hla_low_res_8,cardiac,hla_match_drb1_high,pulm_moderate,hla_low_res_10,efs,efs_time
0,N/A - non-malignant indication,No,NaN,No,NaN,NaN,No TBI,No,6.0,Bone marrow,...,NaN,Unrelated,"N/A, Mel not given",8.0,No,2.0,No,10.0,0.0,42.356
1,Intermediate,No,Intermediate,No,2.0,8.0,"TBI +- Other, >cGy",No,6.0,Peripheral blood,...,Permissive mismatched,Related,"N/A, Mel not given",8.0,No,2.0,Yes,10.0,1.0,4.672
2,N/A - non-malignant indication,No,NaN,No,2.0,8.0,No TBI,No,6.0,Bone marrow,...,Permissive mismatched,Related,"N/A, Mel not given",8.0,No,2.0,No,10.0,0.0,19.793
3,High,No,Intermediate,No,2.0,8.0,No TBI,No,6.0,Bone marrow,...,Permissive mismatched,Unrelated,"N/A, Mel not given",8.0,No,2.0,No,10.0,0.0,102.349
4,High,No,NaN,No,2.0,8.0,No TBI,No,6.0,Peripheral blood,...,Permissive mismatched,Related,MEL,8.0,No,2.0,No,10.0,0.0,16.223


In [8]:
test

,ID,dri_score,psych_disturb,cyto_score,diabetes,hla_match_c_high,hla_high_res_8,tbi_status,arrhythmia,hla_low_res_6,...,karnofsky_score,hepatic_mild,tce_div_match,donor_related,melphalan_dose,hla_low_res_8,cardiac,hla_match_drb1_high,pulm_moderate,hla_low_res_10
0,28800,N/A - non-malignant indication,No,NaN,No,NaN,NaN,No TBI,No,6.0,...,90.0,No,NaN,Unrelated,"N/A, Mel not given",8.0,No,2.0,No,10.0
1,28801,Intermediate,No,Intermediate,No,2.0,8.0,"TBI +- Other, >cGy",No,6.0,...,90.0,No,Permissive mismatched,Related,"N/A, Mel not given",8.0,No,2.0,Yes,10.0
2,28802,N/A - non-malignant indication,No,NaN,No,2.0,8.0,No TBI,No,6.0,...,90.0,No,Permissive mismatched,Related,"N/A, Mel not given",8.0,No,2.0,No,10.0


## Analyse of the Data

### Check the proposition of missing data

In [9]:
missing = np.round(train1.isna().sum()/len(train), 3) * 100
missing

dri_score                  0.5
psych_disturb              7.2
cyto_score                28.0
diabetes                   7.4
hla_match_c_high          16.0
hla_high_res_8            20.2
tbi_status                 0.0
arrhythmia                 7.6
hla_low_res_6             11.4
graft_type                 0.0
vent_hist                  0.9
renal_issue                6.6
pulm_severe                7.4
prim_disease_hct           0.0
hla_high_res_6            18.3
cmv_status                 2.2
hla_high_res_10           24.9
hla_match_dqb1_high       18.1
tce_imm_match             38.7
hla_nmdp_6                14.6
hla_match_c_low            9.7
rituximab                  7.5
hla_match_drb1_low         9.2
hla_match_dqb1_low        14.6
prod_type                  0.0
cyto_score_detail         41.4
conditioning_intensity    16.6
ethnicity                  2.0
year_hct                   0.0
obesity                    6.1
mrd_hct                   57.6
in_vivo_tcd                0.8
tce_matc

In [10]:
df_missing = pd.DataFrame(missing, columns=['values']).reset_index()

In [11]:
# mark different variables which has different category of missing data percentage:

# function to differentiate different category percentage of missing data
def color_map(percent):
  cmap = []
  for x in percent:
    if x >= 20:
      temp = 'background-color: red'
    elif x >= 5:
      temp = 'background-color: yellow'
    else:
      temp = 'background-color: green'
    cmap.append(temp)
  return cmap
# df_missing.style.map(color_map)
df_missing.style.apply(lambda x: color_map(df_missing['values']), subset= ['index','values'], axis = 0)

,index,values
0,dri_score,0.500000
1,psych_disturb,7.200000
2,cyto_score,28.000000
3,diabetes,7.400000
4,hla_match_c_high,16.000000
5,hla_high_res_8,20.200000
6,tbi_status,0.000000
7,arrhythmia,7.600000
8,hla_low_res_6,11.400000
9,graft_type,0.000000


### 1.process missing data, 2.use dummy to transfer category variables to numerical variables, 3.deceide which features import to efs and efs_time 

In [12]:
# 1 find variables which has percentage of missing data less than 5%
indexless5 = np.where(missing < 5)
missingless5 = missing.index[indexless5]
print('variables which has percentage of missing data less than 5% :', missingless5)

# # 2 find variables which has percentage of missing data 5% - 20%
# index5to20 = np.where((missing>= 5) & (missing < 20))
# missing5to20 = missing.index[index5to20]
# print('variables which has percentage of missing data 5% - 20%: ', missing5to20)

# # 3 find variables which has percentage of missing data  more than 20%
# index20 = np.where((missing>= 20))
# missing20 = missing.index[index20]
# print('variables which has percentage of missing data more than 20% :', missing20)


variables which has percentage of missing data less than 5% : Index(['dri_score', 'tbi_status', 'graft_type', 'vent_hist',
       'prim_disease_hct', 'cmv_status', 'prod_type', 'ethnicity', 'year_hct',
       'in_vivo_tcd', 'age_at_hct', 'gvhd_proph', 'sex_match', 'race_group',
       'comorbidity_score', 'karnofsky_score', 'donor_related',
       'melphalan_dose', 'efs', 'efs_time'],
      dtype='object')


In [13]:
# replace missing data in variables which has less than 5 % of missing data with mode or mean 
misssing_less5_object = train1.loc[:, missingless5].select_dtypes(include = 'object').columns
for col in misssing_less5_object:
  train1[col] = train1[col].fillna(train1[col].mode()[0])

misssing_less5_numerical = train1.loc[:, missingless5].select_dtypes(exclude = 'object').columns
for col in misssing_less5_numerical:
  train1[col] = train1[col].fillna(train1[col].mean())

train1[missingless5].isnull().sum()
# check weather there are missing data ln variables which has less than 5 percentage of missing data originally 

dri_score            0
tbi_status           0
graft_type           0
vent_hist            0
prim_disease_hct     0
cmv_status           0
prod_type            0
ethnicity            0
year_hct             0
in_vivo_tcd          0
age_at_hct           0
gvhd_proph           0
sex_match            0
race_group           0
comorbidity_score    0
karnofsky_score      0
donor_related        0
melphalan_dose       0
efs                  0
efs_time             0
dtype: int64

In [14]:
# train12 = train1.copy()

In [15]:
# # for variables which has 5% to 20% of missing data, create a missing indicator
# for col in missing5to20:
#   train12[col +'_missing'] = train1[col].isnull().astype(int)

In [16]:
# Use Cox Proportional Hazards Model to determine whether the missing data has an impact on survival
from lifelines import CoxPHFitter

In [17]:
# # Penalized Cox models, such as Ridge or Lasso regression, can help mitigate issues caused by collinearity and missing data. In lifelines, you can use the penalizer argument to apply regularization to the Cox model.
# cph = CoxPHFitter(penalizer=0.1)
# # use new data train2 (which is train1 remove missing20)
# train2 = train12.copy()
# train2 = train2.drop(columns = missing20)

# # replace category nan with missing, and replace numerical nan with median 
# object_columns = train2.select_dtypes(include = object).columns
# train2[object_columns]= train2.select_dtypes(include = object).fillna('Missing')

# numerical_columns = train2.select_dtypes(exclude = object).columns
# train2[numerical_columns] = train2[numerical_columns].fillna(train2[numerical_columns].median())

# # because Cox Proportional Hazards Model only cope with numerical variables, so change category to numerical(get_dummies)
# train2 = pd.get_dummies(train2, drop_first= True, dtype = int)
# # remove gvhd_proph_FK+- others(not MMF,MTX) this columns in train2, because this columns all have the same value
# train2 = train2.drop(columns = ['gvhd_proph_FK+- others(not MMF,MTX)'])

# # standardize train2, train2 chnage to train
# from sklearn.preprocessing import StandardScaler
# std = StandardScaler()
# train2 = train2.astype(float)
# train2.iloc[:,:] = std.fit_transform(train2.iloc[:,:])

In [18]:
# # use train2 to fit the model 
# cph.fit(train2, duration_col= 'efs_time', event_col= 'efs')
# cph.print_summary()
# # the result of the model is moderate , beacuse the C- index is 0.62, so it need to improve the data quality

###### group variables with missing data less than 5% and more than 5%

In [19]:
# # Use Cox Proportional Hazards Model to determine whether the missing data has an impact on survival
# from lifelines import CoxPHFitter
# # 1 find variables which has percentage of missing data more than 5%
# indexmore5 = np.where(missing>= 5)
# missingmore5 = missing.index[indexmore5]
# print('variables which has percentage of missing data more than 5%: ', missingmore5)

In [20]:
# traintemp = train1.copy()

In [21]:
# for variables which has more than 5% of missing data, create a missing indicator
# for col in missingmore5:
#   traintemp[col +'_missing'] = train1[col].isnull().astype(int)

In [22]:
# # Penalized Cox models, such as Ridge or Lasso regression, can help mitigate issues caused by collinearity and missing data. In lifelines, you can use the penalizer argument to apply regularization to the Cox model.
# cph = CoxPHFitter(penalizer=0.1)
# # use new data train2 (which is train1 remove missing20)
# train3 = traintemp.copy()

# # replace category nan with missing, and replace numerical nan with median 
# object_columns = train3.select_dtypes(include = object).columns
# train3[object_columns]= train3.select_dtypes(include = object).fillna('Missing')

# numerical_columns = train3.select_dtypes(exclude = object).columns
# train3[numerical_columns] = train3[numerical_columns].fillna(train3[numerical_columns].median())

# # because Cox Proportional Hazards Model only cope with numerical variables, so change category to numerical(get_dummies)
# train3 = pd.get_dummies(train3, drop_first= True, dtype = int)
# # remove gvhd_proph_FK+- others(not MMF,MTX) this columns in train3, because this columns all have the same value
# train3 = train3.drop(columns = ['gvhd_proph_FK+- others(not MMF,MTX)'])

# # standardize train3, train3 chnage to train
# from sklearn.preprocessing import StandardScaler
# std = StandardScaler()
# train3 = train3.astype(float)
# train3.iloc[:,:] = std.fit_transform(train3.iloc[:,:])

In [23]:
# # use train3 to fit the model 
# cph.fit(train3, duration_col= 'efs_time', event_col= 'efs')
# cph.print_summary()
# # the result of the model is moderate , beacuse the C- index is 0.63, so it need to improve the data quality

### Plot all variables 

In [24]:
# import seaborn as sns
# import matplotlib.pyplot as plt

# # get numerical and object variables
# numerical_variables = train1.select_dtypes(exclude = object).columns
# object_variables = train1.select_dtypes(include = object).columns

# num_numerical_variables = len(numerical_variables)
# num_object = len(object_variables)

# # plot the boxplot of all numerical variables 
# fig, axes = plt.subplots(6, 4, figsize = (20, 20))
# axes = axes.flatten()
# for i, j in enumerate(numerical_variables):
#   sns.histplot(data = train1, x = j, ax = axes[i])
# plt.show()

In [25]:
# # plot the plot of all numerical variables vs efs
# fig, axes = plt.subplots(6, 4, figsize = (20, 20))
# axes = axes.flatten()
# for i, j in enumerate(numerical_variables[0:-2]):
#   sns.histplot(data = train1, x = j, hue = 'efs', multiple= 'dodge' , ax = axes[i])
# for i in range(-2, 0):
#   axes[i].axis('off')
# plt.show()

# # plot the plot of all numerical variables vs efs_time

# fig, axes = plt.subplots(6, 4, figsize = (20, 20))
# axes = axes.flatten()
# for i, j in enumerate(numerical_variables[0:-2]):
#   sns.scatterplot(data = train1, x = j, y = 'efs_time', hue = 'efs' , ax = axes[i])
# for i in range(-2, 0):
#   axes[i].axis('off')
# plt.show()

In [26]:
# # check the plot of all category variables
# fig, axes = plt.subplots(7, 5, figsize = (20, 25))
# axes = axes.flatten()
# for i, j in enumerate(object_variables):
#   sns.histplot(data = train1, x = j, ax = axes[i])
# plt.show()

In [27]:
# # check the plot of all category variables vs efs and efs_time

# fig, axes = plt.subplots(7, 5, figsize = (20, 25))
# axes = axes.flatten()
# for i, j in enumerate(object_variables):
#   sns.histplot(data = train1, x = j, hue = 'efs', ax = axes[i])
# plt.show()


# fig, axes = plt.subplots(7, 5, figsize = (20, 25))
# axes = axes.flatten()
# for i, j in enumerate(object_variables):
#   sns.scatterplot(data = train1, x = j, y = 'efs_time', hue = 'efs', ax = axes[i])
# plt.show()


#### We can see some variables with missing data are very important, so we are going to use random forests for survival to handle missing values directly

In [87]:
train1.isnull().mean().round(3)

hla_match_c_high                     0.160
hla_high_res_8                       0.202
hla_low_res_6                        0.114
hla_high_res_6                       0.183
hla_high_res_10                      0.249
                                     ...  
melphalan_dose_N/A, Mel not given    0.000
cardiac_Not done                     0.000
cardiac_Yes                          0.000
pulm_moderate_Not done               0.000
pulm_moderate_Yes                    0.000
Length: 150, dtype: float64

In [88]:
# use get_dummy to change category to numerical variables
train1 = pd.get_dummies(train1, drop_first= True, dtype = float)

# split train14 into train4 and test4
from sklearn.model_selection import train_test_split
train4, test4 = train_test_split(train1, train_size= 0.7, random_state= 1, shuffle= True, stratify= train1['efs'])
train4 = train4.reset_index(drop = True).astype(np.float32)
test4 = test4.reset_index(drop = True)

In [89]:
import gc
gc.collect()

from sksurv.ensemble import RandomSurvivalForest
from sksurv.util import Surv

# split X and y from train4
train4_X = train4.drop(columns= ['efs', 'efs_time']).astype(np.float32)
train4_y = train4['efs_time']
train4_r = train4['efs']

# use Surv to create survival object
surv_object = Surv.from_arrays( train4_r, train4_y)

# create and fit the random survival forest model
rsf = RandomSurvivalForest(n_estimators= 10, max_depth= 10, n_jobs = 1, max_features= 'sqrt', random_state= 1)
rsf.fit(train4_X, surv_object)

RandomSurvivalForest(max_depth=10, n_estimators=10, n_jobs=1, random_state=1)

In [91]:
# after training the model, force garbage collection
import gc
gc.collect()
c_index = rsf.score(train4_X, surv_object)
c_index

0.6919570532422651

In [92]:
test4_X = test4.drop(columns = ['efs', 'efs_time'])
test4_y = test4['efs_time']
test4_r = test4['efs']
# calculate the C-index for test data
y_test = Surv.from_arrays(test4_r, test4_y)
c_test_index = rsf.score(test4_X, y_test)
c_test_index

0.6460427419474594

In [46]:
import gc
gc.collect()
from sklearn.inspection import permutation_importance

# access the feature importance 
# because train4_X is quite big data, use smaller data from train4_X
small_train4_X = train4_X.sample(frac = 0.1, random_state= 1)
small_surv_object = surv_object[small_train4_X.index]
feature_importances = permutation_importance(rsf, small_train4_X, small_surv_object, n_repeats= 3, n_jobs = 1 , random_state= 1)

### There are negative permutation importance, whihc means that shuffle these variables improve the modelcompare the original data, this is maybe because the variables are irrelevant, noisy or correlated with other variables, Let check the feature correlation

In [76]:
# give me any variables whose features_importances over average feature_importance > 0 for all variables
feature_importancesbig0 = feature_importances.importances_mean > 0
mean_importancebig0 = feature_importances.importances_mean[feature_importancesbig0].mean()

important_variables = [j for i, j in enumerate(train4_X.columns) if (feature_importances.importances_mean[i] > mean_feature_importance) ]


In [90]:
important_variables

['hla_high_res_8',
 'hla_high_res_10',
 'hla_match_dqb1_high',
 'hla_match_dqb1_low',
 'year_hct',
 'donor_age',
 'age_at_hct',
 'comorbidity_score',
 'karnofsky_score',
 'hla_low_res_10',
 'dri_score_Intermediate',
 'dri_score_N/A - pediatric',
 'psych_disturb_Yes',
 'cyto_score_Poor',
 'tbi_status_TBI + Cy +- Other',
 'tbi_status_TBI +- Other, <=cGy',
 'graft_type_Peripheral blood',
 'pulm_severe_Yes',
 'prim_disease_hct_ALL',
 'prim_disease_hct_IEA',
 'prim_disease_hct_IIS',
 'cmv_status_-/+',
 'prod_type_PB',
 'cyto_score_detail_Intermediate',
 'cyto_score_detail_Poor',
 'conditioning_intensity_NMA',
 'conditioning_intensity_RIC',
 'in_vivo_tcd_Yes',
 'gvhd_proph_FK+ MMF +- others',
 'gvhd_proph_FK+ MTX +- others(not MMF)',
 'sex_match_F-M',
 'sex_match_M-F',
 'sex_match_M-M',
 'race_group_Native Hawaiian or other Pacific Islander',
 'donor_related_Related',
 'cardiac_Yes']

In [77]:
# use new important variables as new dataset
train6_X = train4_X[important_variables]
train6_X 

,hla_high_res_8,hla_high_res_10,hla_match_dqb1_high,hla_match_dqb1_low,year_hct,donor_age,age_at_hct,comorbidity_score,karnofsky_score,hla_low_res_10,...,conditioning_intensity_RIC,in_vivo_tcd_Yes,gvhd_proph_FK+ MMF +- others,gvhd_proph_FK+ MTX +- others(not MMF),sex_match_F-M,sex_match_M-F,sex_match_M-M,race_group_Native Hawaiian or other Pacific Islander,donor_related_Related,cardiac_Yes
0,8.0,10.0,2.0,2.0,2018.0,30.212000,9.235000,3.0,100.0,10.0,...,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
1,NaN,NaN,NaN,NaN,2016.0,NaN,10.757000,0.0,90.0,NaN,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0
2,NaN,NaN,1.0,1.0,2015.0,63.349998,61.537998,0.0,80.0,5.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0
3,8.0,10.0,2.0,2.0,2012.0,36.415001,61.178001,4.0,90.0,10.0,...,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,8.0,10.0,2.0,2.0,2008.0,65.568001,32.032001,0.0,80.0,10.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20155,8.0,10.0,2.0,2.0,2011.0,40.830002,14.493000,3.0,90.0,10.0,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0
20156,8.0,9.0,1.0,2.0,2008.0,24.098000,34.615002,3.0,80.0,10.0,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0
20157,7.0,8.0,1.0,1.0,2014.0,47.189999,38.459000,4.0,90.0,7.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
20158,8.0,10.0,2.0,2.0,2017.0,20.048000,38.620998,2.0,90.0,10.0,...,1.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0


In [78]:
# # calculate VIF (Variance Inflation Factor) 
# from statsmodels.stats.outliers_influence import variance_inflation_factor
# from statsmodels.tools import add_constant

# # use train4_X(efs and efs_time are already removed) to calculate VIF


# # calculate VIF for each feature (because there are some missing data, so can't use VIF directly, so need to remove all missing data)


# train4_X_dropna = train4_X.dropna()

# indicater = 1

# while indicater:
#   vif = [variance_inflation_factor(exog = train4_X_dropna, exog_idx = i) for i in range(len(train4_X_dropna.columns))]
 
#   remain_features = train4_X_dropna.columns
#   if max(vif) > 5: 
#     index_max = np.where(vif == max(vif))
#     column_max = train4_X_dropna.columns[index_max]
#     train4_X_dropna = train4_X_dropna.drop(columns= column_max )
#   else: 
#     indicater = 0

# vif_feature = pd.DataFrame(data = {'remain_features': remain_features, 'vif' : vif})
# vif_feature

In [79]:
# # let check the multicolinarity of variables in the train4_X_dropna
# train5_X_dropna = train4_X.dropna()
# train5_X_dropna_const = add_constant(train5_X_dropna)
# vif = [variance_inflation_factor(exog = train5_X_dropna_const, exog_idx = i) for i in range(len(train5_X_dropna_const.columns))]
# vif_variables = pd.DataFrame(data = {'vaiables':train5_X_dropna_const.columns, 'vif': vif})
# vif_variables.head(149)

In [80]:
# # check whihc variables are deleted from train4_X.columns and also look at features_importance of variables which are depleted 
# for i, j in enumerate(train4_X.columns):
#   if i in remain_features:
#     continue
#   else:
#     print(f'{j} is deleted, and its features_importance is : {feature_importances.importances_mean[i]}')

In [95]:
# create and fit the random survival forest model with new dataset

rsf = RandomSurvivalForest(n_estimators= 30, max_depth= 10, n_jobs = 1, random_state= 1)
rsf.fit(train6_X, surv_object)

RandomSurvivalForest(max_depth=10, n_estimators=30, n_jobs=1, random_state=1)

In [97]:
# calculate the c-index for the new train dataset
gc.collect()
c_index = rsf.score(train6_X, surv_object)
c_index

MemoryError: Unable to allocate 4.41 GiB for an array with shape (20160, 14670, 2) and data type float64

In [94]:
# calculate the C-index for new test data
test6_X = test4_X[important_variables]
y_test = Surv.from_arrays(test4_r, test4_y)
c_test_index = rsf.score(test6_X, y_test)
c_test_index

ValueError: The feature names should match those that were passed during fit.
Feature names seen at fit time, yet now missing:
- arrhythmia_Not done
- arrhythmia_Yes
- cardiac_Not done
- cmv_status_+/-
- cmv_status_-/-
- ...


### Use random forests, XGBoost, or LightGBM to handle missing data, or Cox Proporttional Hazzrds model 